In [5]:
import scipy.io
import numpy as np
from sklearn.metrics import accuracy_score
import time
import matplotlib.pyplot as plt

def rff_gaussian(X, n_components, gamma):
    n_features = X.shape[1]
    W = np.random.normal(0, np.sqrt(2 * gamma), (n_features, n_components))
    b = np.random.uniform(0, 2 * np.pi, n_components)
    Z = np.sqrt(2.0 / n_components) * np.cos(X.dot(W) + b)
    return Z

def rffkmsm(data, n_dims, sigma):
    standardized_data = data 

    # Map the data using RFF
    gamma = sigma  # Set the gamma parameter for the Gaussian kernel
    n_components = n_dims  # Set the number of random features for RFF
    Z = rff_gaussian(standardized_data, n_components, gamma)

    # Calculate the approximate kernel matrix
    approx_kernel_matrix = Z.dot(Z.T)

    # Center the approximate kernel matrix
    row_mean = np.mean(approx_kernel_matrix, axis=0)
    col_mean = np.mean(approx_kernel_matrix, axis=1)
    matrix_mean = np.mean(approx_kernel_matrix)
    centered_approx_kernel_matrix = approx_kernel_matrix - row_mean - col_mean[:, np.newaxis] + matrix_mean

    # Compute the eigenvalues and eigenvectors of the centered approximate kernel matrix
    _, V = np.linalg.eigh(centered_approx_kernel_matrix)
    # Step 5: Select the top k eigenvectors
    return V[:, -n_dims:]

def canonical_similarity(ref_subspaces, input_subspace):
    """
    Computes similarity between reference subspaces and an input subspace based on
    canonical angles between subspaces.
    Args:
        ref_subspaces (3d-array): A 3d-array of reference subspaces of shape
            (NUM_OF_CLASSES x NUM_OF_FEATURES x NUM_OF_DIMS).
        input_subspace (2d-array): A 2d-array representing basis vectors of input subspace.
    Returns:
        A list of similarities (cosines) between 0 and 1 based on canonical angles.
    """
    gramians = ref_subspaces.transpose(0, 2, 1) @ input_subspace
    cosines = [canonical_angles(g) for g in gramians]

    return cosines


def canonical_angles(gramian):
    """
    Computes cosines of canonical angles based on a X.T @ Y gramian matrix, where
    X is a tensor of all refererence subspaces and Y is an input subspace.
    Args:
        gramian (3d-array): A 3d-array of gramians shaped (NUM_OF_CLASSES x
            NUM_OF_DIMS x NUM_OF_DIMS), where NUM_OF_DIMS is the number of subspace
            dimensions.
    Returns:
        A list of cosines of canonical angles.
    """
    cosines = (gramian * gramian).sum() / min(gramian.shape)
    # cosines = np.linalg.norm(g)  # Frobenius norm, more straight-forward way to implement

    return cosines

# utilities 

def split_train_test(data, test_ratio=0.2):
    num_people = data.shape[5]
    num_test = int(num_people * test_ratio)
    test_data = data[:, :, :, :, :, :num_test]
    train_data = data[:, :, :, :, :, num_test:]
    return train_data, test_data

def reshape_data(data, n_samples):
    data = data.reshape(24, 24, 60*7, 30, n_samples)
    data = np.transpose(data, (0, 1, 4, 2, 3))
    data = np.reshape(data, (24, 24, 420 * n_samples, 30))
    return data

def split_and_reshape(data):
    data = np.split(data, 30, axis=3)
    data = [arr.squeeze(axis=3) for arr in data]
    return data

def flatten_and_transpose(data):
    data = [arr.reshape(24 * 24, arr.shape[2]) for arr in data]
    data = [arr.T for arr in data]
    return data

def plot_first_images(first_images):
    grid_rows = 6
    grid_cols = 5
    fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(12, 12))

    for i, img in enumerate(first_images):
        row = i // grid_cols
        col = i % grid_cols
        ax = axes[row, col]
        ax.imshow(img, cmap='gray')
        ax.set_title(f'Class {i+1}')
        ax.axis('off')

    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    plt.show()


start_time = time.time()
data = scipy.io.loadmat('../data/TsukubaHandSize24x24.mat')
data = data['data']

train_data, test_data = split_train_test(data)
train_data = reshape_data(train_data, 80)
test_data = reshape_data(test_data, 20)

train_data = split_and_reshape(train_data)
test_data = split_and_reshape(test_data)

# plot train images
# first_images = [cls_arr[:, :, 0] for cls_arr in X_train]
# plot_first_images(first_images)

# # plot test images
# first_images = [cls_arr[:, :, 0] for cls_arr in X_test]
# plot_first_images(first_images)

train_data = flatten_and_transpose(train_data)
test_data = flatten_and_transpose(test_data)

print('Number of train classes ',len(train_data))
# print('Shape of train class ',train_data[0].shape)
print('Numver of test classes ',len(test_data))
# print('Shape of test class ', test_data[0].shape)


# Fianl preparation of the dataset
NUM_OF_CLASSES = 30
NUM_OF_SETS = 5

X_train = [_X.T for _X in train_data]  # transpose into (N=896, M=30)

test_sets = []

for test_class in test_data:
    test_sets.extend(np.array_split(test_class, NUM_OF_SETS))

X_test = [_X.T for _X in test_sets]
y_test = []

for i, _ in enumerate(range(NUM_OF_CLASSES)):
    y_test.extend(NUM_OF_SETS * [i])  # add corresponding labels into y_test

gamas = [0.001, 0.01, 1, 10, 100, 1000, 10000]

for gama in gamas:
    # Creating the reference subspace
    n_dims = 20
    ref_subspaces = [rffkmsm(_X, n_dims=n_dims, sigma=gama) for _X in X_train]
    ref_subspaces = np.array(ref_subspaces)
    # print(f"Reference subspaces tensor shape: {ref_subspaces.shape}")


    # Create the input subpsace
    input_subspaces = [rffkmsm(set, n_dims=n_dims, sigma=gama) for set in X_test]
    # print(f"Input subspaces type: {type(input_subspaces)}")
    # print(f"Input subspaces len: {len(input_subspaces)}")
    # Calculate the similarities of the input and the reference subspaces
    similarities = np.array(
        [
            canonical_similarity(ref_subspaces, input_subspace)
            for input_subspace in input_subspaces
        ]
    )
    predictions = np.argmax(similarities, axis=1)

    acc = accuracy_score(y_test, predictions)
    end_time = time.time()

    elapsed_time = end_time - start_time
    minutes, seconds = divmod(elapsed_time, 60)

    # print("Time took: {:0>2}:{:05.2f}".format(int(minutes), seconds))
    print(f"MSM accuracy: {acc * 100:.2f}")

Number of train classes  30
Numver of test classes  30
MSM accuracy: 1.33
MSM accuracy: 1.33
MSM accuracy: 2.67
MSM accuracy: 4.67
MSM accuracy: 4.00
MSM accuracy: 7.33
MSM accuracy: 3.33
